In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None
from sklearn.preprocessing import Imputer

In [3]:
df_X_train = pd.read_csv('dengue_features_train.csv', index_col=None)

In [22]:
df_X_train.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,3772 days,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,3779 days,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,3786 days,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,3793 days,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,3800 days,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [5]:
df_y_train = pd.read_csv('dengue_labels_train.csv', index_col=None)

In [6]:
df_y_train.head()

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


In [9]:
regr = RandomForestRegressor(max_depth=30, random_state=0)

In [14]:
X_train = df_X_train
X_train['week_start_date'] = pd.to_datetime(X_train['week_start_date'])
y_train = df_y_train['total_cases']

In [21]:
X_train['week_start_date'] = X_train['week_start_date'] - pd.to_datetime('1980-01-01')

In [31]:
X_train['week_start_date'] = pd.to_numeric(X_train['week_start_date'])/100000000000

In [33]:
df_city = pd.get_dummies(X_train['city'])

In [34]:
X_train_new = pd.concat([X_train, df_city], axis=1)

In [36]:
X_train_new = X_train_new.drop(['city'], axis=1)

In [42]:
X_train_new = X_train_new.fillna(X_train_new.mean())

In [43]:
regr.fit(X_train_new, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [44]:
df_X_test = pd.read_csv('dengue_features_test.csv', index_col=None)

In [46]:
X_test = df_X_test

In [48]:
X_test['week_start_date'] = pd.to_datetime(X_test['week_start_date'])
X_test['week_start_date'] = X_test['week_start_date'] - pd.to_datetime('1980-01-01')
X_test['week_start_date'] = pd.to_numeric(X_test['week_start_date'])/100000000000
df_city_test = pd.get_dummies(X_test['city'])
X_test_new = pd.concat([X_test, df_city_test], axis=1)
X_test_new = X_test_new.drop(['city'], axis=1)
X_test_new = X_test_new.fillna(X_test_new.mean())

In [49]:
result = regr.predict(X_test_new)

In [52]:
result_columns = df_X_test[['city','year','weekofyear']]
result_cases = pd.DataFrame(result)
result_cases.columns = ['total_cases']
result_df = pd.concat([result_columns, result_cases], axis=1)

In [56]:
result_df['total_cases'] = result_df['total_cases'].round().astype(int)

In [57]:
result_df.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,3
1,sj,2008,19,4
2,sj,2008,20,6
3,sj,2008,21,9
4,sj,2008,22,12


In [58]:
result_df.to_csv('rnd_forest.csv', index=False)